In [15]:
import numpy as np
import aplpy
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams, style
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import astropy.io.fits as pyfits
from astropy import units as u
import pyregion
from astropy.wcs import WCS
from sys import argv
import time
import os
import copy
import warnings

In [16]:
#----------------------------------------
# define functions
#----------------------------------------
def brightness_T(I, bmaj, bmin, freq):
    '''
    Calculate the Brightness Teperature (K)
    '''
    I_mJy_beam = I*1000.    # (mJy/beam)
    bmaj_arcs = bmaj*3600.  # (arcsec)
    bmin_arcs = bmin*3600.  # (arcsec)
    freq_GHz = np.divide(freq,1e9)   # (GHz)
    return 1.222e3*np.divide( I_mJy_beam, np.power(freq_GHz,2)*bmaj_arcs*bmin_arcs )

In [17]:
#---------------------------------------
# Input some parameters from parameter.ipynb
#---------------------------------------
%run parameter.ipynb

#---------------------------------------
# Input parameters used in this program:

###-----------
# Path name
# path_image, path_BT

###-----------
# Path name
# outname_b4,  outname_b7

###-----------
# criteria and rms for the imaging
# I_clip_b4, I_clip_b7 [Jy beam-1]

10.0
1.38e+11
5.55555558269e-05 2.77777768219e-05 8.07838836272e-07
0.099999996559
0.0050000000000004
9.9999996558976
10.0
3001
52.2936458333 31.2254166667
2020-03-27T12:13:00.832346


In [18]:
########################################################
# Main code
########################################################
if __name__ == "__main__":
    # Star measure time
    start_time = time.time()
    
    #---------------------------------------
    #set some parameters
    outnames = [
                 outname_b4, outname_b7
                ]
    
    I_clips = [
                I_clip_b4, I_clip_b7
            ]

    i=-1
    for outname in outnames:
        i+=1
        I_image = '%s%s_I.image.fits'%(path_image, outname)

        # Get the cube and header
        I_data = pyfits.getdata(I_image)
        I_hd = pyfits.getheader(I_image)

        # Get the frequency and beam information from header
        hd_freq = float(I_hd['RESTFRQ'])
        hd_bmaj = float(I_hd['BMAJ'])
        hd_bmin = float(I_hd['BMIN'])
        hd_bpa = float(I_hd['BPA'])
        print ("{0} frequency (GHz) = {1}".format(outname[12:18], hd_freq/1e9))
        print ("{0} beam (arcs,arcs,deg)= {1},{2},{3}".format(outname[12:18], hd_bmaj*3600.,hd_bmin*3600.,hd_bpa))

        # Calculate brightness temperature
        BT = brightness_T(I_data, hd_bmaj, hd_bmin, hd_freq)
        BT[I_data<I_clips[i]] = np.nan
        pyfits.writeto('%s%s_BT.fits'%(path_BT, outname), BT, I_hd, overwrite=True)

    # End measure time
    elapsed_time = time.time() - start_time
    print ('#----------------------------------------')
    print ("Done! Time spending = ", elapsed_time, "seconds.")

B4_uv2 frequency (GHz) = 138.0
B4_uv2 beam (arcs,arcs,deg)= 0.200000000977,0.099999996559,8.07838836272e-07
B7_uv2 frequency (GHz) = 336.5
B7_uv2 beam (arcs,arcs,deg)= 0.200000000408,0.0999999950905,-2.98451570302e-06
#----------------------------------------
('Done! Time spending = ', 0.6391370296478271, 'seconds.')
